In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

spark = SparkSession.builder.appName("Jupyter").getOrCreate()

26/01/15 21:40:44 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# Create a demo namespace and table (idempotent if the table already exists)
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.db")
spark.sql("CREATE TABLE IF NOT EXISTS iceberg.db.users (id BIGINT, name STRING, created_at TIMESTAMP) USING iceberg")
spark.sql("SELECT * FROM iceberg.db.users LIMIT 5").show()

+---+----+----------+
| id|name|created_at|
+---+----+----------+
+---+----+----------+



### 1. List the datasets

In [3]:
import os
os.getcwd()
os.listdir("/home/iceberg/data/")

['yellow_tripdata_2021-08.parquet',
 'yellow_tripdata_2022-02.parquet',
 'yellow_tripdata_2022-04.parquet',
 'yellow_tripdata_2022-01.parquet',
 'yellow_tripdata_2021-11.parquet',
 'yellow_tripdata_2021-10.parquet',
 'yellow_tripdata_2021-06.parquet',
 'yellow_tripdata_2021-04.parquet',
 'yellow_tripdata_2022-03.parquet',
 'yellow_tripdata_2021-09.parquet',
 'yellow_tripdata_2021-07.parquet',
 'yellow_tripdata_2021-05.parquet',
 'yellow_tripdata_2021-12.parquet',
 'nyc_film_permits.json']

In [ ]:
data/yellow_tripdata_2025-01.parquet

In [6]:
events = spark.read.option("header", "true").parquet("/home/iceberg/data/yellow_tripdata_2021-05.parquet")
events.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-05-01 00:37:18|  2021-05-01 00:41:07|            2.0|          0.7|       1.0|                 N|         141|         263|           1|        5.0|  3.0|    0.5|       2.

In [4]:
events = spark.read.option("header", "true").csv("/home/iceberg/data/events.csv")
events.show()

+-----------+----------+--------+--------------------+----------+--------------------+
|    user_id| device_id|referrer|                host|       url|          event_time|
+-----------+----------+--------+--------------------+----------+--------------------+
| 1037710827| 532630305|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|
|  925588856| 532630305|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|
|-1180485268| 532630305|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|
|-1044833855| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|
|  747494706| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-26 16:03:...|
|  747494706| 532630305|    NULL|admin.zachwilson....|         /|2021-02-21 16:08:...|
| -824540328| 532630305|    NULL|admin.zachwilson....|         /|2021-09-28 17:23:...|
| -824540328| 532630305|    NULL|    www.eczachly.com|         /|2021-09-29 01:22:...|
| 1833036683| 532630305|    NULL|admin.zach

In [5]:
type(events)

pyspark.sql.dataframe.DataFrame

In [7]:
df = spark.read.option("header", "true").parquet("/home/iceberg/data/yellow_tripdata_2025-01.parquet")
# df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2025-01.parquet")
# df = pd.read_parquet("/home/iceberg/data/yellow_tripdata_2025-01.parquet")

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/home/iceberg/data/yellow_tripdata_2025-01.parquet.

In [7]:
df.show(5, truncate=False)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|1       |2025-01-01 00:18:38 |2025-01-01 00:26:59  |1              |1.6          |1         |N                 |229         |237 

In [8]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)



In [9]:
df.count()

3475226

In [10]:
df.groupBy("VendorId").count().show(8, truncate=False)

+--------+-------+
|VendorId|count  |
+--------+-------+
|1       |753671 |
|7       |1206   |
|2       |2719860|
|6       |489    |
+--------+-------+



### 2. Read files from minio - object storage

In [11]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2025-01.parquet")
# df = pd.read_parquet("/home/iceberg/data/yellow_tripdata_2025-01.parquet")

In [13]:
companies = spark.read.option("header","true").option("inferSchema","true").csv(
    "s3a://warehouse/raw/companies.csv"
)

reviews = spark.read.option("header","true").option("inferSchema","true").csv(
    "s3a://warehouse/raw/reviews.csv"
)


26/01/15 21:33:17 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://warehouse/raw/companies.csv.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveR

Py4JJavaError: An error occurred while calling o86.csv.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(Dat